In [1]:
#number average molecular weight vs extent of reaction with two monomer - interactive with added features
#also added plot for viscosity vs number average molecular weight
#also added a plot for viscosity vs extent of reaction
#viscosity is based on the mark houwink equation [η = K * Mₙ ^ a]
#using plotly to export properly to html - using widgets and added additional features
#a is around 3.4 for entangled polymers and 1 for unentangled ones
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

#monomer molecular weight (g/mol) for 2 monomers
M1_input = widgets.BoundedFloatText(
    value=100.0, min=0, max=10000.0, step=1.0,
    description='Molecular wt. M1 (g/mol):', style={'description_width': 'initial'})#molecular weight of monomer 1
M2_input = widgets.BoundedFloatText(
    value=100.0, min=0, max=10000.0, step=1.0,
    description='Molecular wt. M2 (g/mol):', style={'description_width': 'initial'})#molecular weight of monomer 2

#weight taken for both monomers(g)
m1_input = widgets.BoundedFloatText(
    value=100.0, min=0, max=10000.0, step=1.0,
    description='Weight taken m1 (g):', style={'description_width': 'initial'})#weight taken of monomer 1
m2_input = widgets.BoundedFloatText(
    value=100.0, min=0, max=10000.0, step=1.0,
    description='Weight taken m2 (g):', style={'description_width': 'initial'})#weight taken of monomer 2

#stiochiometric imbalance
r_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.01,
    description='Stiochiometric ratio (r):', continuous_update=True,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px'))
#creates a slider for vertical marker to check for Mn at given p
p_slider = widgets.FloatSlider(
    value=0.5, min=0.0, max=0.99, step=0.01,
    description='Extent of reaction (p):', continuous_update=True,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px'))

# input for the Mark-Houwink constants 
K_input = widgets.BoundedFloatText(value=1e-4, min=1e-10, max=10, step=1e-5, description='Mark-Houwink const.(K):', style={'description_width': 'initial'})
a_input = widgets.BoundedFloatText(value=0.7, min=0.1, max=2.0, step=0.01, description='Mark-Houwink const.(a):', style={'description_width': 'initial'})

# Grouping all the widgets
ui = widgets.VBox([
    widgets.HBox([M1_input, M2_input]),
    widgets.HBox([m1_input, m2_input]),
    r_slider,
    widgets.HBox([K_input, a_input]),
    p_slider])

# Outputs all the plots
output = widgets.Output()

#Function to perform calculations
def update_plot(change=None):
    with output:
        clear_output()

        # Read input values
        M1 = M1_input.value
        M2 = M2_input.value
        m1 = m1_input.value
        m2 = m2_input.value
        r = r_slider.value
        K = K_input.value
        a = a_input.value
        p_marker = p_slider.value
        
        try:
            n1 = m1 / M1#moles of 1 
            n2 = m2 / M2#moles of 2
            f1 = n1 / (n1 + n2)#mole fraction of 1 
            f2 = n2 / (n1 + n2)#mole fraction of 2

            p = np.linspace(0.0, 0.99, 500)
            X_n = (1 + r) / (1 + r - 2 * r * p)#degree of polymerizaiton
            M_n = ((f1 * M1 + f2 * M2) * X_n) / (f1 + f2)#number average molecular weight
            eta = K * M_n**a#viscosity
            
            # Interpolated values at selected p
            M_n_marker = np.interp(p_marker, p, M_n)
            eta_marker = K * M_n_marker**a
            
            #Plot 1: Mₙ vs p with marker
            fig1 = go.Figure()
            fig1.add_trace(go.Scatter(x=p, y=M_n, mode='lines', name='Mₙ vs p', line=dict(color='blue')))
            fig1.add_trace(go.Scatter(x=[p_marker], y=[M_n_marker], mode='markers+text',
                                      name=f'Mₙ at p={p_marker}', marker=dict(color='red', size=10),
                                      text=[f'{M_n_marker:.2f}'], textposition='top center'))#for the p marker line
            fig1.update_layout(
                title="Number Average Molecular Weight vs Extent of Reaction (p)",
                xaxis_title="Extent of Reaction (p)",
                yaxis_title="Number Average Molecular Weight (Mₙ) [g/mol]",
                template="plotly_white")

            #Plot 2: Viscosity vs Mₙ
            fig2 = go.Figure()
            fig2.add_trace(go.Scatter(x=M_n, y=eta, mode='lines', name='[η] vs Mₙ', line=dict(color='green')))
            fig2.add_trace(go.Scatter(x=[M_n_marker], y=[eta_marker], mode='markers+text',
                                      name='Viscosity at Mₙ', marker=dict(color='red', size=10),
                                      text=[f'{eta_marker:.4f}'], textposition='top center'))
            fig2.update_layout(
                title="Intrinsic Viscosity vs Molecular Weight",
                xaxis_title="Number Average Molecular Weight (Mₙ) [g/mol]",
                yaxis_title="Intrinsic Viscosity [η]",
                template="plotly_white")

            #plot 3 - Viscosity vs extent of reaction
            fig3 = go.Figure()
            fig3.add_trace(go.Scatter(x=p, y=eta, mode='lines', name='[η] vs p', line=dict(color='red')))
            fig3.add_trace(go.Scatter(x=[p_marker], y=[eta_marker], mode='markers+text',
                                      name='Viscosity at p', marker=dict(color='blue', size=10),
                                      text=[f'{eta_marker:.4f}'], textposition='top center'))
            fig3.update_layout(
                title="Intrinsic Viscosity vs Extent of Reaction",
                xaxis_title="Extent of Reaction (p)",
                yaxis_title="Intrinsic Viscosity [η]",
                template="plotly_white")
            # Display plots
            fig1.show()
            fig2.show()
            fig3.show()
            #displaying data 
            display(Markdown(f"""**Mole Fractions :** $f_1$ = {f1:.3f}, $f_2$ = {f2:.3f}"""))
            display(Markdown(f"""**At selected p = {p_marker:.2f}**, $M_n$ = {M_n_marker:.2f} g/mol- $[\eta]$ = {eta_marker:.4f}"""))
            display(Markdown(f"""The torque is calculated using the Mark Houwink Equation - $[η = K * (Mₙ) ^ a]$"""))
            
        except Exception as e:
            print(f"Error: {e}")

# Observe changes
for widget in [M1_input, M2_input, m1_input, m2_input, r_slider, K_input, a_input, p_slider]:
    widget.observe(update_plot, names='value')

# Display widgets and output
display(ui, output)
update_plot()

Output()